In [ ]:
# ARTIST INFO CODIGO LIMPIO 25 FEB 2025

In [1]:
import pandas as pd
import requests
import json
import re  # Importar el módulo para limpiar HTML

/Users/isabelgarcia/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
api_key = "XXXXXXXXXXXXXXXXXXX"
share_secret = "XXXXXXXXXXXXXXXXXXXXX"
artist_name= "Chanel"

- Sacamos informacion del artista desde last.fm y nos lo muestra en un DataFrame.
- Información extraida: nombre, url, reproducciones, artistas similares, generos y resumen de la biografia.

In [5]:
#  Lista de los 5 artistas más populares de la playlist de Eurovisión 2022
lista_artistas_eurovision = ["Chanel", "Loreen", "Måneskin", "Sam Ryder", "Käärijä"]

In [ ]:
# Primero hago la funcion para un unico artista: 

def obtener_info_artista(artist_name, api_key):  #esta funcion para sacar info del artista con ARTIST_NAME y API_KEY: devuelve un diccionario con sus datos.


    # Endpoint de la API de Last.fm
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json"
    
    # Hago la llamada a la API
    response = requests.get(url)  # hago la llamada a la api.
    info_artist = response.json()  # y la respuesta de la llamada la convierto en json.

    # compruebo la respuesta de la API
    if "error" in info_artist:
        print(f"Error en la API: {info_artist['message']}")
        return pd.DataFrame()   # si hay error dime el error y dame un DATAFRAME vacio.

    # información del artista
    nombre = info_artist["artist"].get("name")
    url = info_artist["artist"].get("url") 
    oyentes = info_artist["artist"]["stats"].get("listeners")
    reproducciones = info_artist["artist"]["stats"].get("playcount")

    #resumen de la biografía y limpiar etiquetas HTML
    bio_summary = info_artist["artist"]["bio"].get("summary")
    bio_summary_clean = re.sub(r"<.*?>", "", bio_summary)  # Eliminar etiquetas HTML

    # géneros musicales
    generos = [tag["name"] for tag in info_artist["artist"]["tags"]["tag"][:3]] # limito a tres generos del artista
    generos_texto = ", ".join(generos)  # utilizo este codigo para que aparezca como cadena de texto y no como lista con corchetes y comillas.

    #artistas similares
    artistas_similares = [artista["name"] for artista in info_artist["artist"]["similar"]["artist"][:3]]  # limito a tres artistas similares
    artistas_similares_texto = ", ".join(artistas_similares)   # igual que en genero lo convierto en cadena de texto.

    return {                    # me devuelve un diccionario con todos los datos que le he pedido del artista.
        "artista": nombre,
        "url":url,
        "oyentes": oyentes,
        "reproducciones": reproducciones,
        "generos": generos_texto,
        "artistas_similares": artistas_similares_texto,
        "biografia_resumen": bio_summary_clean
    }


#  la información de VARIOS artistas y la almacena en un DataFrame.

def obtener_info_varios_artistas(lista_artistas, api_key):
   
    datos_artistas = []

    for artista in lista_artistas:
        info = obtener_info_artista(artista, api_key)  # llamo a la funcion para obtener los datos para cada artista, que itere
        if info:  # Solo agregar si la consulta fue ok
            datos_artistas.append(info)

    
    df_artistas = pd.DataFrame(datos_artistas)   # Creo DataFrame con todos los artistas
    return df_artistas   # return el DataFrame de todos los artistas.
    
"""df_artistas_eurovision = obtener_info_varios_artistas(lista_artistas_eurovision, api_key)

print(df_artistas_eurovision.head())"""  #lo inhabilito porque es otra forma de sacar el dataframe pero se ve peor.

# DataFrame con la información de todos los artistas truco de CHATGPT
from IPython.display import display  # Importar display para Jupyter

df_artistas_eurovision = obtener_info_varios_artistas(lista_artistas_eurovision, api_key)
display(df_artistas_eurovision)  # ✅ Mejor visualización en Jupyter



,artista,url,oyentes,reproducciones,generos,artistas_similares,biografia_resumen
0,Chanel,https://www.last.fm/music/Chanel,196029,3146736,"House, female vocalists, chillout","Kaleen, Alessandra, Loreen","Chanel Terrero Martinez (born July 29, 1991 in..."
1,Loreen,https://www.last.fm/music/Loreen,678496,16718104,"pop, swedish, female vocalists","Dotter, Cornelia Jakobs, Anna Bergendahl",Loreen (born Lorine Zineb Nora Talhaoui on Oct...
2,Måneskin,https://www.last.fm/music/M%C3%A5neskin,1484050,78926261,"rock, glam rock, italian","Palaye Royale, The Pretty Reckless, Mahmood","Måneskin (Italian pronunciation: ˈmɔːneskin, D..."
3,Sam Ryder,https://www.last.fm/music/Sam+Ryder,135531,1689904,"pop, british, Eurovision","Subwoolfer, KEiiNO, Mae Muller","Sam Ryder (born 1989 Maldon, England) is a Bri..."
4,Käärijä,https://www.last.fm/music/K%C3%A4%C3%A4rij%C3%A4,223049,8414818,"finnish, rap, Eurovision","KUUMAA, Bess, Portion Boys","Jere Pöyhönen, known professionally as Käärijä..."


In [ ]:
# Para convertirlo a CSV, no se si es necesario, lo inhabilito.

"""df_artistas_eurovision.to_csv("artistas_eurovision_2022.csv", index=False)
print("✅ Archivo guardado: artistas_eurovision_2022.csv")"""